In [1]:
from line_solver import *

import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = np.empty(3, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.FCFS)
node[2] = Queue(model, 'Queue2', SchedStrategy.FCFS)

jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 10, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 10, node[0], 0)

node[0].set_service(jobclass[0], Exp.fitMean(1.0))  # mean = 1
node[1].set_service(jobclass[0], Exp.fitMean(1.5))  # mean = 1.5
node[2].set_service(jobclass[0], Exp.fitMean(3.0))  # mean = 3.0

node[0].set_service(jobclass[1], Exp.fitMean(1.0))  # mean = 1
node[1].set_service(jobclass[1], Exp.fitMean(1.5))  # mean = 1.5
node[2].set_service(jobclass[1], Exp.fitMean(3.0))  # mean = 3.0

P = model.init_routing_matrix()
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)
P.set(jobclass[0], jobclass[0], node[1], node[2], 1.0)
P.set(jobclass[0], jobclass[0], node[2], node[0], 1.0)

P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)
P.set(jobclass[1], jobclass[1], node[1], node[2], 1.0)
P.set(jobclass[1], jobclass[1], node[2], node[0], 1.0)

model.link(P)

In [3]:
# Solve with MVA only (following MATLAB example)
options = SolverMVA.defaultOptions()
solver = np.array([], dtype=object)
solver = np.append(solver, SolverMVA(model, options))

AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'SOLVER: {solver[s].get_name()}')
    AvgTable[s] = solver[s].get_avg_table()

SOLVER: SolverMVA
MVA analysis [method: default(exact), lang: java, env: 21.0.8] completed in 0.020706s.
  Station JobClass    QLen    Util    RespT   ResidT    ArvR    Tput
0   Delay   Class1  0.1667  0.1667   1.0000   1.0000  0.1667  0.1667
1   Delay   Class2  0.1667  0.1667   1.0000   1.0000  0.1667  0.1667
2  Queue1   Class1  0.5000  0.2500   3.0000   3.0000  0.1667  0.1667
3  Queue1   Class2  0.5000  0.2500   3.0000   3.0000  0.1667  0.1667
4  Queue2   Class1  9.3333  0.5000  56.0001  56.0001  0.1667  0.1667
5  Queue2   Class2  9.3333  0.5000  56.0001  56.0001  0.1667  0.1667
